<a href="https://colab.research.google.com/github/jsmackie/TalkingToWALS/blob/mainline/TalkingToWALS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
    cohere \
    langchain \
    tiktoken \
    pinecone-client \
    langchain-openai \
    langchain-pinecone
!wget https://raw.githubusercontent.com/jsmackie/TalkingToWALS/mainline/HidingBehindWALS.py -O HidingBehindWALS.py
import HidingBehindWALS as wals
agent = wals.WALS()

In [8]:
agent.TalkToWALS('Summarize Chapter 1')

Chapter 1 of the World Atlas of Language Structures (WALS) covers the feature of "Consonant Inventories" in languages. It provides information on the number and types of consonants that languages make use of, ranging from smaller inventories to larger ones. The chapter helps to understand the variety and distribution of consonant sounds across languages worldwide.
